In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [35]:
class Node: 
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None) -> None:
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf_node(self):
        return self.value is not None

class CustomDecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_features=None) -> None:
        self.min_samples_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
    
    def _most_common_label(self, y: np.ndarray):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def _entropy(self, y: np.ndarray):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _split(self, x_column: np.ndarray, threshold):
        left_idxs = np.argwhere(x_column <= threshold).flatten()
        right_idxs = np.argwhere(x_column > threshold).flatten()
        return left_idxs, right_idxs

    def _information_gain(self, y: np.ndarray, x_column: np.ndarray, threshold):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(x_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
        
        ig = parent_entropy - child_entropy
        return ig

    def _best_split(self, x: np.ndarray, y: np.ndarray, feat_idxs: np.ndarray):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            x_column = x[:, feat_idx]
            thresholds = np.unique(x_column)
            for threshold in thresholds:
                gain = self._information_gain(y, x_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold

        return split_idx, split_threshold

    def _grow_tree(self, x: np.ndarray, y: np.ndarray, depth: int=0):
        n_samples, n_features = x.shape
        n_labels = len(np.unique(y))
        
        # Check the stopping criteria
        if(depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Find the best splits
        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)
        best_feat, best_thresh = self._best_split(x, y, feat_idxs)
        
        # Create child nodes
        left_idxs, right_idxs = self._split(x[:, best_feat], best_thresh)
        
        # Repeat for child nodes
        left = self._grow_tree(x[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(x[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def fit(self, x: np.ndarray, y: np.ndarray):
        self.n_features = x.shape[1] if not self.n_features else min(self.n_features, x.shape[1])
        self.root = self._grow_tree(x, y)
    
    def _traverse_tree(self, x, node: Node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def predict(self, X: np.ndarray):
        return np.array([self._traverse_tree(x, self.root) for x in X])

In [43]:
data = load_breast_cancer()
x, y = data.data, data.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [44]:
custom_model = CustomDecisionTree()
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
custom_model.fit(x_train, y_train)
model_predictions = model.predict(x_test)
custom_model_predictions = custom_model.predict(x_test)

In [45]:
print("Custom Decision Tree Classifier")
print(confusion_matrix(y_test, custom_model_predictions))
print("Sklearn Decision Tree Classifier")
print(confusion_matrix(y_test, model_predictions))

Custom Decision Tree Classifier
[[39  4]
 [ 4 67]]
Sklearn Decision Tree Classifier
[[40  3]
 [ 3 68]]
